In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

# Create a zip file object
with zipfile.ZipFile("/kaggle/input/working-on-tur/_output_.zip", "r") as zip_obj:
    # Extract all the files to a directory
    zip_obj.extractall("/kaggle/working/")

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import os
import shutil
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import glob
import cv2
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
np.random.seed(42)

In [ ]:
selected_words = os.listdir('/kaggle/working/Dataset/frames/Train')

In [ ]:
words = os.listdir('/kaggle/working/Dataset/frames/Train')
label_map = {label:num for num, label in enumerate(words)}
print(label_map)

In [ ]:
def preprocess_data(data_path,split,f_avg):

    # Initialize the lists of sequences and labels
    sequences = []
    labels = []
    
    # Iterate over the words
    for word in tqdm(words):
        word_path = os.path.join(data_path, split, word)
        word_id = os.path.basename(word_path).zfill(4)
    
        lh_keypoints_folder = os.path.join(word_path, "lh_keypoints")
        rh_keypoints_folder = os.path.join(word_path, "rh_keypoints")
        pose_keypoints_folder = os.path.join(word_path, "pose_keypoints")
    
        # Iterate through the sequences (numpy arrays) contained in the lh_keypoints folder
        for sequence in os.listdir(lh_keypoints_folder):
            # Load the left hand array
            res_lh = np.load(os.path.join(lh_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_lh.shape[0], f_avg)
            res_lh = res_lh[:num_frames, :]
            while num_frames < f_avg:
                res_lh = np.concatenate((res_lh, np.expand_dims(res_lh[-1, :], axis=0)), axis=0)
                num_frames += 1
    
            # Load the right hand array
            res_rh = np.load(os.path.join(rh_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_rh.shape[0], f_avg)
            res_rh = res_rh[:num_frames, :]
            while num_frames < f_avg:
                res_rh = np.concatenate((res_rh, np.expand_dims(res_rh[-1, :], axis=0)), axis=0)
                num_frames += 1
    
            # Load the pose array
            res_pose = np.load(os.path.join(pose_keypoints_folder, sequence))
    
            # Determine how many frames to select
            num_frames = min(res_pose.shape[0], f_avg)
            res_pose = res_pose[:num_frames, :]
            while num_frames < f_avg:
                res_pose = np.concatenate((res_pose, np.expand_dims(res_pose[-1, :], axis=0)), axis=0)
                num_frames += 1
    
#             # Append the subsequence to the list of sequences
            sequences.append(np.concatenate((res_pose,res_lh, res_rh), axis=1))
            # Append the label to the list of labels
            labels.append(label_map[word])
    
    # Convert the lists of sequences and labels to numpy arrays
    X = np.array(sequences)
    y = to_categorical(labels).astype(int)
    
    return X, y

In [ ]:
lista = []
toFrames = '/kaggle/working/Dataset/frames'
for fold in os.listdir(toFrames):
    toTrain = os.path.join(toFrames,fold)
    for word in os.listdir(toTrain):
        toWord = os.path.join(toTrain,word)
        for vid in os.listdir(toWord):
            toVid = os.path.join(toWord,vid)
            lista.append(int(len(os.listdir(toVid))))
mapa = {}
for i in lista : 
    mapa[i] = 0
for i in lista:
    mapa[i]+=1
median = []
for k,v in mapa.items():
    if v>=len(words):
        median.append(k)
avg = round(sum(median)/len(median))

avg = np.sort(median)[-1]
avg

In [ ]:
#train and validation splits
data_path = '/kaggle/working/Dataset/npy_arrays'

X_train,y_train=preprocess_data(data_path,'Train/',48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# X_train = np.concatenate(X_train,X_val)
# y_train=np.concatenate(y_train,y_val)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
#test split
X_test,y_test=preprocess_data(data_path,'Test',48)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# # Define the Bidirectional LSTM model with Attention
# model = tf.keras.Sequential([
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
# #     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(64, activation='relu'),
# #     tf.keras.layers.BatchNormalization(),
# #     tf.keras.layers.Dense(64, activation='linear'),
#     tf.keras.layers.Dense(len(words), activation='softmax')
# ])
# # model = tf.keras.Sequential([
# #     tf.keras.layers.Dense(64, input_dim=14),
# #     tf.keras.layers.BatchNormalization(),
# #     tf.keras.layers.Activation('tanh'),
# #     tf.keras.layers.Dropout(0.5),
# #     tf.keras.layers.Dense(64),
# #     tf.keras.layers.BatchNormalization(),
# #     tf.keras.layers.Activation('tanh'),
# #     tf.keras.layers.Dropout(0.5),
# #     tf.keras.layers.Dense(2),
# #     tf.keras.layers.BatchNormalization(),
# #     tf.keras.layers.Activation('softmax')
# # ])
# # model = Sequential()
# # activ = 'relu'
# # model.add(tf.keras.layers.Conv2D(64, (1, 3), strides=(1, 1), padding='same', activation=activ,input_shape=(130, 51, 225)))
# # model.add(tf.keras.layers.Conv2D(64, (1, 3), strides=(1, 1), padding='same', activation=activ))
# # #model.add(BatchNormalization(axis = 3))
# # model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 2) ))

# # model.add(tf.keras.layers.Conv2D(128, (1, 3), strides=(1, 1), padding='same', activation=activ))
# # model.add(tf.keras.layers.Conv2D(128, (1, 3), strides=(1, 1), padding='same', activation=activ))
# # model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 2)))

# # model.add(tf.keras.layers.Conv2D(256, (1, 3), strides=(1, 1), padding='same', activation=activ))
# # model.add(tf.keras.layers.Conv2D(256, (1, 3), strides=(1, 1), padding='same', activation=activ ))
# # model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 2)))
# # model.add(tf.keras.layers.Dropout(.5))
# # A = X_train.shape
# # model.add(tf.keras.layers.Flatten())
# # model.add(tf.keras.layers.Dense(int(A[1] * 1/4.), activation=activ))
# # model.add(tf.keras.layers.Dropout(.5))

# # model.add(tf.keras.layers.Dense(len(words), activation='softmax'))

# optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.003)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# # Set up early stopping
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',  # Metric to monitor for early stopping
#     mode='min',  # Set mode to 'min' for minimizing the metric
#     patience=10,  # Number of epochs with no improvement before stopping
#     restore_best_weights=True,  # Restore the best model weights
#     verbose=1
# )

In [ ]:
import tensorflow as ts
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(words)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('softmax')
])
# model = tf.keras.Sequential([
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(len(words), activation='softmax')
# ])
# model = tf.keras.Sequential([
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(64, activation='linear'),
#     tf.keras.layers.BatchNormalization(),
#      tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64, activation='linear'),
#     tf.keras.layers.Dense(len(words), activation='softmax')
# ])
# Compile the model
import tensorflow as ts
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0003)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['categorical_accuracy'])
# Set up early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',  # Metric to monitor for early stopping
    mode='max',  # Set mode to 'min' for minimizing the metric
    patience=30,  # Number of epochs with no improvement before stopping
    restore_best_weights=True,  # Restore the best model weights
    verbose=1
)
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9 ** (epoch // 10000)

# Create the LearningRateScheduler callback
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

es=tf.keras.callbacks.EarlyStopping( monitor="val_loss", patience=1,
                                     verbose=1,  restore_best_weights=True)
rlronp=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_categorical_accuracy", factor=0.5, patience=3,
                                             verbose=1,mode = 'max')

In [ ]:
# Define the schedule function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9 ** (epoch // 10000)

# Create the LearningRateScheduler callback
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# es=tf.keras.callbacks.EarlyStopping( monitor="val_loss", patience=1,
#                                      verbose=1,  restore_best_weights=True)
# rlronp=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=3,
#                                              verbose=1,mode = 'min')

In [ ]:
model_training_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=[early_stopping,rlronp,callback])

In [ ]:
model_evaluation_history = model.evaluate(X_train, y_train)

In [ ]:
model_evaluation_history = model.evaluate(X_test, y_test)


In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):


    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()
# Visualize the training and validation loss metrices.
plot_metric(model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(model_training_history, 'categorical_accuracy', 'val_categorical_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
#Predicted sign
res = model.predict(X_test)
words[np.argmax(res[1])]

In [ ]:
#Real sign
words[np.argmax(y_test[1])]

In [ ]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

In [ ]:
yhat = model.predict(X_test)
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

y = []
for v in ytrue:
    y.append(get_key_by_value(label_map, v))
print(y)

In [ ]:
ypred = []
for v in yhat:
    ypred.append(get_key_by_value(label_map, v))
print(ypred)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Assuming y and ypred are your target labels and predicted labels, respectively

y_subset = y
ypred_subset = ypred

# Get unique class labels
class_labels = np.unique(y_subset)

# Compute confusion matrix
cm = confusion_matrix(y_subset, ypred_subset, labels=class_labels)
# Create a DataFrame from the confusion matrix
df_cm = pd.DataFrame(cm, index=class_labels, columns=class_labels)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
# Plot the confusion matrix
plt.figure(figsize=(len(words), len(words)*0.8))
sns.set(font_scale=1.3)  # for label size
sns.heatmap(df_cm, cmap="Blues", annot=True, fmt="d", annot_kws={"size": 12})
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'Tur_Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

os.makedirs('/kaggle/working/Model',exist_ok=True)
# Save your Model.
modelPath = f'/kaggle/working/Model/{model_file_name}'
model.save(modelPath)

In [ ]:
# Import the libraries
import tensorflow as tf
from tensorflow import keras

# Load the H5 model
model = keras.models.load_model(modelPath)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("/kaggle/working/TurkishModel.tflite", "wb") as f:
  f.write(tflite_model)